In [1]:
using GeometryTypes
const gt = GeometryTypes
using LCMGL
using Meshing
using DrakeVisualizer
using BenchmarkTools
# using ProfileView

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/lib/v0.5/DrakeVisualizer.ji for module DrakeVisualizer.


adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [12]:
include("gjk.jl")

Gjk

In [19]:
s = Simplex(Vec(2., -1), Vec(2.5, 0), Vec(2., 1), Vec(1., 1), Vec(1., -1))
simplex, best_pt = Gjk.gjk(s)
@show best_pt
simplex._

best_pt = Vec(1.0,0.0)


3-element Array{Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},Void},1}:
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},Void}(Vec(2.0,-1.0),nothing)
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},Void}(Vec(1.0,1.0),nothing) 
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},Void}(Vec(1.0,-1.0),nothing)

In [46]:
x = [1,2,3,4,5]
view(x, [1:3 5:5])

LoadError: LoadError: ArgumentError: number of rows of each array must match (got (3,1))
while loading In[46], in expression starting on line 2

In [14]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
acc = Gjk.AcceleratedMesh(mesh)
simplex, best_pt = Gjk.gjk(acc)
@show best_pt
simplex._

best_pt = Vec(1.2000000000000002,-0.8999999999999999,0.0)


4-element Array{Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64},1}:
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.2,-1.5,-0.6),1)       
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.41522,-0.75,0.3),291) 
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.12286,-1.05,-0.15),45)
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.2,-0.9,0.0),253)      

In [15]:
pt = Vec(0., 0, 0)
md = gt.MinkowskiDifference(acc, pt)
simplex, best_pt = Gjk.gjk(md)
@show best_pt
simplex._

best_pt = Gjk.Difference{FixedSizeArrays.Vec{3,Float64},FixedSizeArrays.Vec{3,Float64},FixedSizeArrays.Vec{3,Float64}}(Vec(1.2000000000000002,-0.8999999999999999,0.0),Vec(1.2000000000000002,-0.8999999999999999,0.0),Vec(0.0,0.0,0.0))


4-element Array{Gjk.Difference{FixedSizeArrays.Vec{3,Float64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}},1}:
 Gjk.Difference{FixedSizeArrays.Vec{3,Float64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}}(Vec(1.2,-1.5,-0.6),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.2,-1.5,-0.6),1),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}(Vec(0.0,0.0,0.0),nothing))             
 Gjk.Difference{FixedSizeArrays.Vec{3,Float64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}}(Vec(1.41522,-0.75,0.3),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64}(Vec(1.41522,-0.75,0.3),291),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}(Vec(0.0,0.0,0.0),nothing))   
 Gjk.Difference{FixedSizeArrays.Vec{3,Float64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Int64},Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Void}}(Vec(1.12286,-1.05,-0.15),Gjk.Tagged{FixedSize

In [17]:
Gjk.value(best_pt)

Vec(1.2000000000000002,-0.8999999999999999,0.0)

In [21]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
simplex = Gjk.gjk(mesh)[1]
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex._
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [22]:
acc = Gjk.AcceleratedMesh(mesh)
simplex, pt_best = Gjk.gjk(acc)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex._
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [23]:
for i = 1:10
    x0 = 4 * (rand(3) .- 0.5)
    mesh = DrakeVisualizer.contour_mesh(x -> sum((x - x0).^2) - 1, [-3, -3, -3], [3, 3, 3])
    Visualizer(mesh)
    acc = Gjk.AcceleratedMesh(mesh)
    simplex, pt_best = Gjk.gjk(acc)
    LCMGLClient("simplex") do lcmgl
        color(lcmgl, 1, 0, 0, 0.5)
        for tagged in simplex._
            sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
        end
        color(lcmgl, 0, 0, 1, 0.5)
        sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
        switch_buffer(lcmgl)
    end
end

In [24]:
@benchmark Gjk.gjk($mesh)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  4.28 kb
  allocs estimate:  140
  minimum time:     14.81 μs (0.00% GC)
  median time:      22.49 μs (0.00% GC)
  mean time:        22.56 μs (1.91% GC)
  maximum time:     4.37 ms (98.55% GC)

In [25]:
@benchmark Gjk.gjk($acc)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     3
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  6.73 kb
  allocs estimate:  177
  minimum time:     8.66 μs (0.00% GC)
  median time:      11.66 μs (0.00% GC)
  mean time:        12.61 μs (8.01% GC)
  maximum time:     1.33 ms (97.68% GC)

In [26]:
simplex, pt_best = Gjk.gjk(mesh)
@benchmark Gjk.gjk($mesh, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     9
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  608.00 bytes
  allocs estimate:  23
  minimum time:     2.26 μs (0.00% GC)
  median time:      3.13 μs (0.00% GC)
  mean time:        3.19 μs (2.59% GC)
  maximum time:     453.77 μs (98.56% GC)

In [27]:
simplex, pt_best = Gjk.gjk(acc)
@benchmark Gjk.gjk($acc, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     174
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  800.00 bytes
  allocs estimate:  26
  minimum time:     637.00 ns (0.00% GC)
  median time:      702.00 ns (0.00% GC)
  mean time:        824.26 ns (12.76% GC)
  maximum time:     22.59 μs (95.30% GC)